In [9]:
import pandas as pd
import numpy as np
import feather
import pickle
import pickle5
import re
import sqlite3
import geopandas as gpd

# note pandarallel works well on mac but has issue with windows
# see requirements for windows  - https://github.com/nalepae/pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# connect to the database
# note: connects to/creates a db file with the name in the quotes if does not exist
con = sqlite3.connect('streetsofnyc.db')
cur = con.cursor()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Load LION - Street Data

In [2]:
lion = gpd.read_file("Data/LION/LION.shp")

# note some address contain '-' we only want the portion before the dash

lion['l_lowadd'] = lion['LLo_Hyphen'].str.split('-').str[0].astype(float)
lion['l_highadd'] = lion['LHi_Hyphen'].str.split('-').str[0].astype(float)
lion['r_lowadd'] = lion['RLo_Hyphen'].str.split('-').str[0].astype(float)
lion['r_highadd'] = lion['RHi_Hyphen'].str.split('-').str[0].astype(float)

# get combined lower and upper limit for street segment

lion['r_lowadd'].replace({'0':np.nan},inplace=True)
lion['l_lowadd'].replace({'0':np.nan},inplace=True)
lion['c_lowadd']= lion[['l_lowadd','r_lowadd']].min(axis=1,skipna=True)
lion['c_highadd']=lion[['l_highadd','r_highadd']].max(axis=1)

lion.head(5)

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,SpecAddr,FaceCode,SeqNum,StreetCode,SAFStreetC,LGC1,LGC2,LGC3,LGC4,LGC5,LGC6,LGC7,LGC8,LGC9,BOE_LGC,SegmentID,SegCount,LocStatus,LZip,RZip,LBoro,RBoro,L_CD,R_CD,LATOMICPOL,RATOMICPOL,LCT2010,LCT2010Suf,RCT2010,RCT2010Suf,LCB2010,LCB2010Suf,RCB2010,RCB2010Suf,LCT2000,LCT2000Suf,RCT2000,RCT2000Suf,LCB2000,LCB2000Suf,RCB2000,RCB2000Suf,LCT1990,LCT1990Suf,RCT1990,RCT1990Suf,LAssmDist,LElectDist,RAssmDist,RElectDist,SplitElect,LSchlDist,RSchlDist,SplitSchl,LSubSect,RSubSect,SanDistInd,MapFrom,MapTo,BoroBndry,MH_RI_Flag,XFrom,YFrom,XTo,YTo,ArcCenterX,ArcCenterY,CurveFlag,Radius,NodeIDFrom,NodeIDTo,NodeLevelF,NodeLevelT,ConParity,Twisted,RW_TYPE,PhysicalID,GenericID,NYPDID,FDNYID,LBlockFace,RBlockFace,LegacyID,Status,StreetWidt,StreetWi_1,StreetWi_2,BikeLane,BIKE_TRAFD,ACTIVE_FLA,POSTED_SPE,Snow_Prior,Number_Tra,Number_Par,Number_Tot,Carto_Disp,FCC,ROW_Type,LLo_Hyphen,LHi_Hyphen,RLo_Hyphen,RHi_Hyphen,FromLeft,ToLeft,FromRight,ToRight,Join_ID,L_PD_Servi,R_PD_Servi,TRUCK_ROUT,Shape__Len,geometry,l_lowadd,l_highadd,r_lowadd,r_highadd,c_lowadd,c_highadd
0,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,None,2510,03070,226700,None,01,None,None,None,None,None,None,None,None,1,0078126,1,X,10456,10456,2.000,2.000,203,203,402,101,149,None,185,None,3001,None,2000,None,149,None,137,None,4000,None,1000,None,149,None,137,None,79,040,79,040,None,09,09,None,1B,1B,None,3D,3D,None,None,1010964,241812,1011265,241555,0,0,None,0,0047740,9045677,M,M,None,None,1,35231.000,30694.000,None,None,1422600653,1422602017,0078126,2,34.000,34.000,None,None,None,None,25,S,2,None,4,None,None,None,599,699,596,716,599,699,596,716,2251001000000,None,None,None,396.031,"LINESTRING (-73.90347 40.83036, -73.90238 40.8...",599.000,699.000,596.000,716.000,596.000,716.000
1,2,WEST 192 STREET,None,0,U,None,B,None,A,DOT,None,7984,00040,274810,None,01,None,None,None,None,None,None,None,None,1,0079796,1,None,10468,10468,2.000,2.000,207,207,302,104,265,None,265,None,2000,None,1004,None,265,None,265,None,3001,None,1003,None,265,None,265,None,78,045,78,059,None,10,10,None,1A,1A,None,3C,3C,None,None,1011577,255024,1011335,255164,0,0,None,0,0048679,0048678,M,M,None,None,1,35248.000,30711.000,None,None,1522607129,1522607721,0079796,2,30.000,30.000,None,None,None,None,25,S,1,None,3,None,None,None,58,98,63,99,58,98,63,99,2798401000000,None,None,None,279.361,"LINESTRING (-73.90120 40.86662, -73.90207 40.8...",58.000,98.000,63.000,99.000,58.000,99.000
2,3,UNION AVENUE,None,0,U,None,B,None,W,DOT,None,7280,00130,270420,None,01,None,None,None,None,None,None,None,None,1,0077356,4,X,10459,10459,2.000,2.000,203,203,402,401,135,None,131,None,2000,None,3006,None,135,None,131,None,4000,None,4001,None,135,None,131,None,79,046,79,026,None,12,12,None,1A,1A,None,6C,6C,None,None,1011601,239640,1011786,240230,0,0,None,0,0047288,0047822,M,M,None,None,1,35252.000,30715.000,None,None,1422603726,1422604132,0077356,2,34.000,34.000,None,None,None,None,25,S,1,None,3,None,None,None,1017,1079,1016,1084,1017,1079,1016,1084,2728001000000,None,None,None,618.327,"LINESTRING (-73.90118 40.82440, -73.90051 40.8...",1017.000,1079.000,1016.000,1084.000,1016.000,1084.000
3,4,UNION AVENUE,BEHAGEN PLAYGROUND COMFORT STA,0,U,None,B,None,W,DOT,X,7280,00130,270420,212795,01,None,None,None,None,None,None,None,None,1,0077356,4,X,10459,10459,2.000,2.000,203,203,402,401,135,None,131,None,2000,None,3006,None,135,None,131,None,4000,None,4001,None,135,None,131,None,79,046,79,026,None,12,12,None,1A,1A,None,6C,6C,None,None,1011601,239640,1011786,240230,0,0,None,0,0047288,0047822,M,M,None,None,1,35252.000,30715.000,None,None,1422603726,1422604132,0077356,2,34.000,34.000,None,None,None,None,25,S,1,None,3,None,None,None,None,None,None,None,0,0,0,0,21279502000000X,None,None,None,618.327,"LINESTRING (-73.90118 40.82440, -73.90051 40.8...",NaN,NaN,NaN,NaN,NaN,NaN
4,5,UNION AVENUE,BEHAGEN PLAYGROUND FIELD NORTH,0,U,None,B,None,W,DOT,X,7280,00130,270420,212

In [3]:
%%time

# create LION table in database from df - note geomerty data not supported in SQLITE
lion.drop(columns='geometry').to_sql('LION',con,if_exists='replace',index=False)

CPU times: user 19.5 s, sys: 1.1 s, total: 20.6 s
Wall time: 23 s


## Load cd_indicators - Community Disctrict Demographics

In [4]:
cd_indicators = pd.read_csv('Data/cd_demographics/cd_indicators_overall.csv')

In [14]:
cd_indicators

,the_geom,cartodb_id,the_geom_webmercator,acres,acs_tooltip,acs_tooltip_2,acs_tooltip_3,area_sqmi,borocd,cb_email,cb_website,cd_full_title,cd_short_title,cd_son_fy2018,cd_tot_bldgs,cd_tot_resunits,count_hosp_clinic,count_libraries,count_parks,count_public_schools,crime_count,crime_count_boro,crime_count_nyc,crime_per_1000,crime_per_1000_boro,crime_per_1000_nyc,female_10_14,female_15_19,female_20_24,female_25_29,female_30_34,female_35_39,female_40_44,female_45_49,female_5_9,female_50_54,female_55_59,female_60_64,female_65_69,female_70_74,female_75_79,female_80_84,female_85_over,female_under_5,fp_100_area,fp_100_bldg,fp_100_cost_burden,fp_100_cost_burden_value,fp_100_mhhi,fp_100_mortg_value,fp_100_openspace,fp_100_ownerocc,fp_100_ownerocc_value,fp_100_permortg,fp_100_pop,fp_100_rent_burden,fp_100_rent_burden_value,fp_100_resunits,fp_100_openspace2,fp_500_bldg,fp_500_cost_burden,fp_500_cost_burden_value,fp_500_mhhi,fp_500_mortg_value,fp_500_openspace,fp_500_ownerocc,fp_500_ownerocc_value,fp_500_permortg,fp_500_pop,fp_500_rent_burden,fp_500_rent_burden_value,fp_500_resunits,lep_rate,lep_rate_boro,lep_rate_nyc,lot_area_commercial_office,lot_area_industrial_manufacturing,lot_area_mixed_use,lot_area_open_space,lot_area_other_no_data,lot_area_parking,lot_area_public_facility_institution,lot_area_res_1_2_family_bldg,lot_area_res_multifamily_elevator,lot_area_res_multifamily_walkup,lot_area_transportation_utility,lot_area_vacant,lots_commercial_office,lots_industrial_manufacturing,lots_mixed_use,lots_open_space,lots_other_no_data,lots_parking,lots_public_facility_institution,lots_res_1_2_family_bldg,lots_res_multifamily_elevator,lots_res_multifamily_walkup,lots_total,lots_transportation_utility,lots_vacant,male_10_14,male_15_19,male_20_24,male_25_29,male_30_34,male_35_39,male_40_44,male_45_49,male_5_9,male_50_54,male_55_59,male_60_64,male_65_69,male_70_74,male_75_79,male_80_84,male_85_over,male_under_5,mean_commute,mean_commute_boro,mean_commute_nyc,moe_bach_deg,moe_bach_deg_boro,moe_bach_deg_nyc,moe_foreign_born,moe_hh_rent_burd,moe_hh_rent_burd_boro,moe_hh_rent_burd_nyc,moe_lep_rate,moe_lep_rate_boro,moe_lep_rate_nyc,moe_mean_commute,moe_mean_commute_boro,moe_mean_commute_nyc,moe_over65_rate,moe_over65_rate_boro,moe_over65_rate_nyc,moe_poverty_rate,moe_under18_rate,moe_under18_rate_boro,moe_under18_rate_nyc,moe_unemployment_nyc,moe_unemployment_boro,moe_unemployment,neighborhoods,over65_rate,over65_rate_boro,over65_rate_nyc,pct_asian_nh,pct_bach_deg,pct_bach_deg_boro,pct_bach_deg_nyc,pct_black_nh,pct_clean_strts,pct_clean_strts_boro,pct_clean_strts_nyc,pct_foreign_born,pct_hh_rent_burd,pct_hh_rent_burd_boro,pct_hh_rent_burd_nyc,pct_hispanic,pct_other_nh,pct_served_parks,pct_white_nh,pop_2000,pop_2010,pop_acs,pop_change_00_10,poverty_rate,poverty_rate_boro,poverty_rate_nyc,puma,shared_puma,shared_puma_cd,son_issue_1,son_issue_2,son_issue_3,total_lot_area,under18_rate,under18_rate_boro,under18_rate_nyc,unemployment_boro,unemployment,unemployment_nyc,v_pluto,v_acs,v_facdb,v_crime
0,NaN,1,NaN,976.300,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.500,101,man01@cb.nyc.gov,www1.nyc.gov/site/manhattancb1/index.page,Manhattan Community District 1,Manhattan CD 1,NaN,1725,45629,10,2,13,19,1055,26271,92480,6.800,16.100,11,1.100,2.400,3.900,7.200,6.500,4.700,3.600,2.900,1.800,2.300,2.400,2.600,2.400,1.800,1.000,0.700,1.200,2.700,0.850,653,0.285,2164,121000,4900,0.048,0.221,7599,0.645,16733,0.353,7367,19756,0.048,1036,0.285,2913,118000,6272,0.040,0.242,10238,0.613,29221,0.350,8750,28097,6.200,15.800,23.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359,4,635,23,17,23,72,34,167,66,1461,36,25,1.400,1.800,2.800,5.700,6.300,5.200,3.600,3.600,1.700,2.700,2.700,2.700,2.000,1.300,0.900,0.700,0.800,2.600,25.400,32.100,41.200,0.900,0.400,0.200,0.900,1.600,0.500,0.300,0.700,0.200,0.100,0.600,0.200,0.100,0.600,0,0,0.700,0.600,0,0,0.100,0.100,0.300,"B

In [5]:
%%time

# create cd_indic table in database from df 
cd_indicators.to_sql('cd_indic',con,if_exists='replace',index=False)

CPU times: user 35 ms, sys: 12.1 ms, total: 47.1 ms
Wall time: 73.4 ms


## Load weather data
Preprocessed in a seperate file by Moutaz

In [11]:
weather=pickle5.load(open('Data/weather.pkl','rb'))

In [12]:
weather

,STATION,NAME,LATITUDE,LONGITUDE,DATE,Precipitation,Snowfall,Snow/depth,Water_equivalent_snow_on_ground,Fog/ice_fog/freezing_fog
0,USC00280907,"BOONTON 1 SE, NJ US",40.892,-74.396,2018-01-01,0.000,0.000,0.000,0.000,0.000
1,USC00280907,"BOONTON 1 SE, NJ US",40.892,-74.396,2018-01-02,0.000,0.000,0.000,0.000,0.000
2,USC00280907,"BOONTON 1 SE, NJ US",40.892,-74.396,2018-01-03,0.000,0.000,0.000,0.000,0.000
3,USC00280907,"BOONTON 1 SE, NJ US",40.892,-74.396,2018-01-04,0.000,0.000,0.000,0.000,0.000
4,USC00280907,"BOONTON 1 SE, NJ US",40.892,-74.396,2018-01-05,0.560,5.000,5.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...
98316,USW00054743,"CALDWELL ESSEX CO AIRPORT, NJ US",40.876,-74.283,2021-04-15,0.220,0.000,0.000,0.000,0.000
98317,USW00054743,"CALDWELL ESSEX CO AIRPORT, NJ US",40.876,-74.283,2021-04-16,0.020,0.000,0.000,0.000,0.000
98318,USW00054743,"CALDWELL ESSEX CO AIRPORT, NJ US",40.876,-74.283,2021-04-17,0.000,0.000,0.000,0.000,0.000
98319,USW00054743,"CALDWELL ESSEX CO AIRPORT, NJ US",40.876,-74.283,2021-04-18,0.000,0.000,0.000,0.000,0.000


In [13]:
%%time

# create weather table in database from df 
weather.to_sql('weather',con,if_exists='replace',index=False)

CPU times: user 392 ms, sys: 98.9 ms, total: 491 ms
Wall time: 560 ms


## Load Collision Data

Preprocessed in a seperate file by Sheila

In [20]:
col20=pd.read_csv('Data/collisions/collision20.csv')
col19=pd.read_csv('Data/collisions/collision19.csv')

col_combined=pd.concat([col20,col19])

# create collision table in database from df 
col_combined.to_sql('collision',con,if_exists='replace',index=False)

## Create LION + Demographics Table
Query create new table LION_DEM that takes key features of Street plus the community demographic data joined on the L_CD in LION

Will need to delete this table if it exist

In [ ]:
%%time

query='''
CREATE TABLE LION_Dem AS
SELECT 
b.OBJECTID,b.Street,b.FeatureTyp,b.SegmentTyp,b.NonPed,b.TrafDir,b.LocStatus,b.LZip,b.RZip,b.LBoro,b.RBoro,
b.L_CD,b.R_CD,b.CurveFlag,b.Radius,b.RW_Type,b.PhysicalID,b.StreetWidt,b.BikeLane,b.BIKE_Trafd,b.Number_Tra,
b.Number_Par,b.Number_Tot,b.Posted_Spe,b.Truck_Rout,b.c_lowadd,b.c_highadd,b.StreetCode,
a.cd_short_title,a.cd_tot_bldgs,a.cd_tot_resunits,a.crime_count,a.crime_per_1000,a.lep_rate,a.lots_commercial_office,a.lots_industrial_manufacturing,
a.lots_mixed_use,a.lots_open_space,a.lots_parking,a.lots_total,a.mean_commute,a.over65_rate,a.under18_rate,
a.pct_bach_deg,a.pct_foreign_born,a.pct_hh_rent_burd,a.pct_white_nh,a.pct_black_nh,(100-a.pct_white_nh) AS pct_non_white_nh,
a.poverty_rate,a.unemployment 
FROM LION b
LEFT OUTER JOIN cd_indic a
ON b.L_CD = a.borocd
'''

con.execute(query)

## Merge Tickets with Streets and Demographics to New Table

Query to create new table ticketstreetdem - combines tickets data with street and demographic details

Will need to delete this table if it exist

In [ ]:
%%time

query='''
CREATE TABLE IF NOT EXISTS ticketstreetdem AS
SELECT a.`Summons Number`,a.`Violation Code`,a.`Clean Violation Des`,a.`Issue Date`,a.`Violation Time`,b.*
FROM tickets a
LEFT OUTER JOIN LION_Dem b
ON a.Street1LU = b.StreetCode
WHERE b.c_lowadd<=a.`House Number Clean`
AND b.c_highadd>=a.`House Number Clean`
'''
con.execute(query)